#### Prediction of missing eye tracking data with neural networks
Lucas D. Haberkamp<sup>1,2,3</sup>, Michael D. Reddix<sup>1</sup>

<sup>1</sup>Naval Medical Research Unit - Dayton  
<sup>2</sup>Oak Ridge Institute for Science and Education  
<sup>3</sup>Leidos   

---

This script generates predictions for the point of gaze with the trained temporal convolutional network model (Script B) and classification predictions with the trained classifier model (Script C)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

In [ ]:
# load the keras models
tcn = tf.keras.models.load_model("../Models/TCN")
classifier = tf.keras.models.load_model("../Models/Classifier")

In [ ]:
# Set paths for data which was used to scale the training dataset
scale_x_path = '../Data/Prep/train/trainx/P01_trainx.csv'
scale_y_path = '../Data/Prep/train/trainy-regression/P01_trainyreg.csv'

In [ ]:
# Create x and y scaler objects
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

# Read the appropriate files (same ones used to fit the scaler objects during training) and fit the scaler objects
scale_x_df = pd.read_csv(scale_x_path)
scale_y_df = pd.read_csv(scale_y_path)

x_scaler.fit(scale_x_df.values)
y_scaler.fit(scale_y_df.values)

In [ ]:
# Create 3D tensors for regression predictions
def createsequences(curr_x):
    x = tf.data.Dataset.from_tensor_slices(curr_x)
    x = x.window(window_size, shift=1, drop_remainder=True)
    x = x.flat_map(lambda w: w.batch(window_size))
    x = x.batch(batch_size).prefetch(1)
    return x

In [ ]:
# path with both training & holdout inputs
test_x_path = '../Data/Prep/all/allx/'

# columns to output in the final DataFrame
out_cols = ['CWI_x', 'CWI_y', 'CWI_z', 'Classification', 
    'p-accelerometer', 'p-airspeed',
    'p-attitude','p-hsi', 'p-altimeter',
    'p-verticalspeed','p-alternates', 'p-standby',
    'p-engine', 'p-parking','p-flap',
    'p-misc-hdd',
    'p-kneeboard', 'p-bugeye']

window_size = 256
batch_size = 256

for i, filename in enumerate(os.listdir(test_x_path)):
    tmp_file = os.path.splitext(filename)[0][:-5]
    print(tmp_file)
    f = os.path.join(test_x_path, filename)

    tmp_df = pd.read_csv(f) # read the current file

    x_scaled = x_scaler.transform(tmp_df.values) # scale the data with MinMaxScaler

    x_test = createsequences(x_scaled) # generate sequences with the scaled data

    pred = tcn.predict(x_test) # predict the point of gaze

    tmp_label = classifier.predict(pred) # classify the predicted point of gaze

    tmp_result = y_scaler.inverse_transform(pred) # inverse transform the predicted point of gaze data

    # as predictions are autoregressive, no predictions are made for the 1st 31 values
    result = np.zeros(shape=(tmp_df.shape[0], 3)) 
    result[window_size-1:,:] = tmp_result 

    label = np.zeros(shape=(tmp_df.shape[0], tmp_label.shape[1]))
    label[window_size-1:,:] = tmp_label

    # the label with the highest probability is assigned as the predicted label
    final_label = np.argmax(label, axis=-1).astype('int')
    final_label = np.expand_dims(final_label,axis=1)

    # concatenate point of gaze, classification, and classification probablities
    output = np.concatenate([result, final_label, label], axis=1)

    # create a DataFrame and write to a .csv for further analysis
    out_df = pd.DataFrame(output, columns=out_cols)
    
    out_df.to_csv('../Data/Predictions/' + tmp_file + "_predicted.csv", index=False)
